In [2]:
instructions = "Instructions: Select an Excel file to start processing URLs. Click 'Good Link' if the current URL is good, 'Bad Link' if it's bad, or 'Skip' to move to the next URL without marking it. Click 'End and Save' to stop processing URLs and save the current state to the Excel file. If you see an email or phone number on the website, enter them into the provided fields, press enter, and continue on. They will be updates in the excel sheet."
import pandas as pd
import webbrowser
import tkinter as tk
from tkinter import filedialog, messagebox, font
from urllib.parse import urlparse

class ToolTip(object):
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tip_window = None

    def show_tip(self):
        x, y, _, _ = self.widget.bbox("insert")
        x += self.widget.winfo_rootx() + 25
        y += self.widget.winfo_rooty() + 25
        self.tip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)
        tw.wm_geometry(f"+{x}+{y}")
        label = tk.Label(tw, text=self.text, background="#ffffe0", relief=tk.SOLID, borderwidth=1, font=("tahoma", "12", "normal"))
        label.pack(ipadx=1)

    def hide_tip(self):
        tw = self.tip_window
        self.tip_window = None
        if tw:
            tw.destroy()

def create_tooltip(widget, text):
    tool_tip = ToolTip(widget, text)
    def enter(event):
        tool_tip.show_tip()
    def leave(event):
        tool_tip.hide_tip()
    widget.bind('<Enter>', enter)
    widget.bind('<Leave>', leave)

class App:
    def __init__(self, root):
        self.root = root
        self.urls = None
        self.current_index = 0
        self.df = None
        self.filepath = None

        self.font = font.Font(size=14)
        self.bold_font = font.Font(size=16, weight='bold')

        self.instructions = tk.Label(root, text=instructions, font=self.font, wraplength=500)
        self.instructions.pack(pady=10)

        self.current_label = tk.Label(root, font=self.bold_font)
        self.current_label.pack(pady=10)

        self.email_label = tk.Label(root, text="Email:", font=self.font)
        self.email_label.pack()
        self.email_entry = tk.Entry(root, font=self.font)
        self.email_entry.pack()

        self.phone_label = tk.Label(root, text="Phone Number:", font=self.font)
        self.phone_label.pack()
        self.phone_entry = tk.Entry(root, font=self.font)
        self.phone_entry.pack()

        self.button_open = tk.Button(root, text="Open File", command=self.open_file, font=self.font)
        self.button_open.pack(pady=10)
        create_tooltip(self.button_open, "Click to select an Excel file and start processing URLs.")

        self.button_good = tk.Button(root, text="Good Link", command=self.mark_good, state=tk.DISABLED, font=self.font)
        self.button_good.pack(pady=10)
        create_tooltip(self.button_good, "Click if the current URL is good. This will mark the current row as 'Good Link' and move to the next URL.")

        self.button_bad = tk.Button(root, text="Bad Link", command=self.mark_bad, state=tk.DISABLED, font=self.font)
        self.button_bad.pack(pady=10)
        create_tooltip(self.button_bad, "Click if the current URL is bad. This will mark the current row as 'Bad Link' and move to the next URL.")

        self.button_skip = tk.Button(root, text="Skip", command=self.skip, state=tk.DISABLED, font=self.font)
        self.button_skip.pack(pady=10)
        create_tooltip(self.button_skip, "Click to skip the current URL without marking it as good or bad. This will move to the next URL.")

        self.next_label = tk.Label(root, font=self.bold_font)
        self.next_label.pack(pady=10)

        self.button_end = tk.Button(root, text="End and Save", command=self.end_and_save, state=tk.DISABLED, font=self.font)
        self.button_end.pack(pady=10)
        create_tooltip(self.button_end, "Click to stop processing URLs and save the current state to the Excel file.")

    def open_file(self):
        self.filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        self.df = pd.read_excel(self.filepath)
        self.urls = self.df.iloc[:, 4]
        self.df['Company Viability'] = ''
        self.current_index = 0
        self.update_buttons_state(tk.NORMAL)
        self.next_url()

    def next_url(self):
        while self.current_index < len(self.urls):
            url = self.urls[self.current_index]
            if pd.notna(url) and self.validate_url(url):  # Add URL validation
                webbrowser.open(url)
                self.current_label['text'] = f"Current: {url}"
                if self.current_index + 1 < len(self.urls):
                    next_url = self.urls[self.current_index + 1]
                    self.next_label['text'] = f"Next: {next_url if pd.notna(next_url) else 'None'}"
                else:
                    self.next_label['text'] = "Next: None"
                break  # Break out of the loop when a valid URL is found
            else:
                self.current_index += 1  # Skip to the next URL if current URL is not valid

        # If no valid URL is found, update the buttons and display the end message
        if self.current_index >= len(self.urls):
            self.update_buttons_state(tk.DISABLED)
            self.current_label['text'] = "No more URLs."
            self.df.to_excel(self.filepath, index=False)
            messagebox.showinfo("Information", "All URLs have been processed. The Excel file has been updated.")

    def mark_good(self):
        self.df.loc[self.current_index, 'Company Viability'] = 'Good Link'
        self.update_email_and_phone()
        self.current_index += 1
        self.next_url()

    def mark_bad(self):
        self.df.loc[self.current_index, 'Company Viability'] = 'Bad Link'
        self.update_email_and_phone()
        self.current_index += 1
        self.next_url()

    def skip(self):
        self.update_email_and_phone()
        self.current_index += 1
        self.next_url()

    def end_and_save(self):
        self.update_buttons_state(tk.DISABLED)
        self.df.to_excel(self.filepath, index=False)
        messagebox.showinfo("Information", "The Excel file has been updated.")

    def update_buttons_state(self, state):
        self.button_good['state'] = state
        self.button_bad['state'] = state
        self.button_skip['state'] = state
        self.button_end['state'] = state

    def update_email_and_phone(self):
        email = self.email_entry.get().strip()
        phone = self.phone_entry.get().strip()
        if email:
            self.df.loc[self.current_index, 'Email'] = email
            self.email_entry.delete(0, tk.END)
        if phone:
            self.df.loc[self.current_index, 'Phone'] = phone
            self.phone_entry.delete(0, tk.END)

    def validate_url(self, url):
        try:
            result = urlparse(url)
            return all([result.scheme, result.netloc])
        except ValueError:
            return False

root = tk.Tk()
app = App(root)
root.mainloop()


In [3]:
!pip install openpyxl